# A Developer's Introduction to `schedview`

## Notebook setup

### Notebook formatting

`lab_black` is nice for notebook development, but remove or comment it out when running the notebook in Times Square.

In [1]:
%load_ext lab_black

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

### Development version of modules

At this stage of the porject, the versions of `schedview` and the scheduler-related modules in the defaul `LSST` envionment at the USDF are usually significantly out of date.

So, to get the latest versions, check out the versions you want, build them, and modify the path to point to them. For example, if your development directories are in `/sdf/data/rubin/user/${USER}/devel`, then you can set the modules you want to load devel versions of, then add the relevant entries to the path:

In [4]:
# To change which devel modules to use, just comment or uncomment relevant elements in this list:
devel_module_names = [
    #    'uranography',
    "rubin_scheduler",
    "rubin_sim",
    "schedview",
]

import os
import sys
import pwd
from pathlib import Path

username = pwd.getpwuid(os.getuid())[0]
devel_module_path = Path("/sdf/data/rubin/user").joinpath(username, "devel")
for module_name in devel_module_names:
    sys.path.insert(0, devel_module_path.joinpath(module_name).as_posix())

### Import necessary modules

In [45]:
import pandas as pd
import warnings
import datetime
import astropy
import astropy.time
import bokeh
import bokeh.io
import schedview
import erfa
import uranography.api
import rubin_sim
import rubin_scheduler
import schedview
import schedview.collect
import schedview.collect.visits
import schedview.plot
from IPython.display import HTML, display, Markdown, HTML

### Supress benign astropy warnings

In simulations, we use dates the astropy finds suspicious, or can't do "proper" sidereal time conversion for. Ignore.

In [6]:
# Degraded IERS accuracy is never going to be important for these figures.

# If IERS degraded accuracy encountered, don't fail, just keep going.
astropy.utils.iers.conf.iers_degraded_accuracy = "ignore"

# Don't even complain.
warnings.filterwarnings(
    "ignore",
    category=astropy.utils.exceptions.AstropyWarning,
    message="Tried to get polar motions for times after IERS data is valid. Defaulting to polar motion from the 50-yr mean for those. This may affect precision at the arcsec level. Please check your astropy.utils.iers.conf.iers_auto_url and point it to a newer version if necessary.",
)

# In simulations, we go far enough into the future that the erfa module finds it "dubious".
# Keep the complaints quiet.
warnings.filterwarnings(
    "ignore",
    category=erfa.ErfaWarning,
    message=r".*dubious year.*",
)

### Support the display of plots in the jupyter notebook

#### Support display of `bokeh` plots

In [7]:
bokeh.io.output_notebook()

Loading BokehJS ...

#### Support display of `matplotlib` plots

In [8]:
%matplotlib inline

## Dealing with dayobs

`schedview` has a tool for dealing with the day of observation, as defined in SITCOMTM-032:

> A natural number representing the observation day (in timezone UTC-12:00) when
the takeImages command began executing. 


In instance of `schedview.DayObs` can be created either from any of several representations of a date:

In [9]:
print(schedview.DayObs.from_date("20241123"))

2024-11-23


In [10]:
print(schedview.DayObs.from_date(20241123))

2024-11-23


In [11]:
print(schedview.DayObs.from_date("2023-11-23"))

2023-11-23


In [12]:
print(schedview.DayObs.from_date(60637, int_format="mjd"))

2024-11-23


You can also create an instance from a time, in which case it will create an instance of `DayObs` for the date on which that time falls.

In [13]:
print(schedview.DayObs.from_time("2024-11-24 08:00:00Z"))

2024-11-23


In [14]:
print(schedview.DayObs.from_time("2024-11-24 04:00:00Z"))

2024-11-23


You can give it a time zone offset, following ISO-8601

In [15]:
print(schedview.DayObs.from_time("2024-11-24 08:00:00-4"))

2024-11-24


In [16]:
print(schedview.DayObs.from_time("2024-11-24 04:00:00-4"))

2024-11-23


You can also use an instance of `astropy.Time`,  or a floating point Modified Julian Date:

In [17]:
t = astropy.time.Time("2024-11-24 08:00:00Z")
print(schedview.DayObs.from_time(t))

2024-11-23


python's `datetime.datetime`:

In [18]:
t = datetime.datetime(2024, 11, 24, 8, 0, 0, tzinfo=datetime.UTC)
print(schedview.DayObs.from_time(t))

2024-11-23


or a floating point MJD (interperted in UTC, such that floating point MJDs early in the date correspend to the integer MJD of the day before):

In [19]:
day_obs = schedview.DayObs.from_time(60638.2)
print(day_obs, day_obs.mjd)

2024-11-23 60637


An instance of `schedview.DayObs` can provide the date in a variety of formats:

In [20]:
day_obs = schedview.DayObs.from_date("2023-11-23")

print("day_obs.yyyymmdd:", day_obs.yyyymmdd, type(day_obs.yyyymmdd))
print("day_obs.mjd:", day_obs.mjd, type(day_obs.mjd))
print("str(day_obs)", str(day_obs), type(str(day_obs)))
print("day_obs.date", day_obs.date, type(day_obs.date))
print("day_obs.start", day_obs.start, type(day_obs.start))
print("day_obs.end", day_obs.end, type(day_obs.end))

day_obs.yyyymmdd: 20231123 <class 'int'>
day_obs.mjd: 60271 <class 'int'>
str(day_obs) 2023-11-23 <class 'str'>
day_obs.date 2023-11-23 <class 'datetime.date'>
day_obs.start 2023-11-23 12:00:00 <class 'astropy.time.core.Time'>
day_obs.end 2023-11-24 12:00:00 <class 'astropy.time.core.Time'>


## Overall architecture of creating a plot

See the [architecture overview page](https://schedview.lsst.io/architecture.html) in the schedview documentation.

## Tables of visits

`schedview` can load tables of visits (represented as `pandas.DataFrame`s) either from the ConsDB or an opsim database. It can also use `maf` stackers (usually defined in `rubin_sim`) to supplement the columns that come directly from those sources.
There are three tools in `schedview` that return such tables:
- `schedview.collect.opsim.read_opsim`, which reads visits from an `opsim` database
- `schedview.collect.consdb.read_consdb`, which queries the consdb.
- `schedview.collect.visits.read_visits` is a more general interface which tries to automatically determine whether to query the `opsim` database for a baseline simulation or the consdb for a specific telescope, and calls either `schedview.collect.opsim.read_opsim` or `schedview.collect.consdb.read_consdb` to read visits *for one night*.

A number of other `schedview` functions expect visit tables in the form returned by these functions. 

### Reading visits from an `opsim` database

`schedview.collect.opsim.read_opsim` uses `rubin_sim.maf.get_sim_data` to load data visit, and returns it as a `pandas.DataFrame`.

In [21]:
visits = schedview.collect.opsim.read_opsim(
    opsim_uri="/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.5/baseline/baseline_v3.5_10yrs.db",
    start_time=astropy.time.Time("2026-01-01 12:00:00Z"),
    end_time=astropy.time.Time("2026-01-02 12:00:00Z"),
)
visits.head()

,fieldRA,fieldDec,observationStartMJD,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,airmass,seeingFwhm500,...,rotTelPos,rotTelPos_backup,moonAz,sunAz,moonRA,moonDec,moonDistance,solarElong,moonPhase,start_date
observationId,,,,,,,,,,,,,,,,,,,,,
151317,27.934791,-32.335155,61042.035190,29.2,i,-21.599124,-21.599124,2,1.027004,0.660279,...,17.073714,16.497145,35.354494,233.928806,81.475155,28.923614,79.584198,90.103458,89.127295,2026-01-02 00:50:40.398234129+00:00
151318,26.567847,-34.857639,61042.035651,29.2,i,-30.135429,-30.135429,2,1.036100,0.660279,...,17.289467,16.497145,35.230981,233.826928,81.480801,28.924655,82.159644,87.937978,89.129593,2026-01-02 00:51:20.217450859+00:00
151319,25.108555,-37.374015,61042.036098,29.2,i,-36.657527,-36.657527,2,1.046410,0.660279,...,17.537149,16.497145,35.110783,233.727753,81.486281,28.925662,84.747834,85.765609,89.131823,2026-01-02 00:51:58.891310688+00:00
151320,26.087071,-42.350891,61042.036607,29.2,i,-52.464087,-52.464087,2,1.056439,0.660279,...,18.034303,16.497145,34.973925,233.614793,81.492501,28.926803,87.627298,84.263910,89.134354,2026-01-02 00:52:42.833216784+00:00
151321,24.422336,-44.884688,61042.037050,29.2,i,-54.953872,-54.953872,2,1.073839,0.660279,...,18.208301,16.497145,34.854479,233.516171,81.497914,28.927794,90.245531,82.089392,89.136556,2026-01-02 00:53:21.105147598+00:00


Keywords not recognized by the `read_opsim` function itself are passed to `rubin_sim.maf.get_sim_data`.
This can be used, for example, to apply `maf` stackers:

In [22]:
visits = schedview.collect.opsim.read_opsim(
    opsim_uri="/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.5/baseline/baseline_v3.5_10yrs.db",
    start_time=astropy.time.Time("2026-01-01 12:00:00Z"),
    end_time=astropy.time.Time("2026-01-02 12:00:00Z"),
    stackers=[rubin_sim.maf.HourAngleStacker()],
)
visits.loc[:, ["observationStartMJD", "fieldRA", "fieldDec", "filter", "HA"]].head()

,observationStartMJD,fieldRA,fieldDec,filter,HA
observationId,,,,,
151317,61042.035190,27.934791,-32.335155,i,1.044353
151318,61042.035651,26.567847,-34.857639,i,1.146574
151319,61042.036098,25.108555,-37.374015,i,1.254632
151320,61042.036607,26.087071,-42.350891,i,1.201637
151321,61042.037050,24.422336,-44.884688,i,1.323280


A few `schedview` tools that take these tables of visits as arguments expect columns created using the stackers defined in `schedview.collect.visits.NIGHT_STACKERS`,
such that the `visits` table should be loaded thus:

In [23]:
visits = schedview.collect.opsim.read_opsim(
    opsim_uri="/sdf/group/rubin/web_data/sim-data/sims_featureScheduler_runs3.5/baseline/baseline_v3.5_10yrs.db",
    start_time=astropy.time.Time("2026-01-01 12:00:00Z"),
    end_time=astropy.time.Time("2026-01-02 12:00:00Z"),
    stackers=schedview.collect.visits.NIGHT_STACKERS,
)

### Reading visits from the ConsDB

Visits for a night or set of nights can also be queried from the ConsDB.
The columns returned include *both* the columns are they are natively named in the ConsDB, *and also* columns that match what `opsim` produces.
The result is redundant table: `fieldRA` and `s_ra` have the same data, for example.
But, this redundancy allows the table to be used where names according to what is in opsim are expected.

In [24]:
visits = schedview.collect.consdb.read_consdb(
    "lsstcomcam",
    stackers=schedview.collect.visits.NIGHT_STACKERS,
    day_obs="2024-11-15",
    num_nights=1,
)
visits.head()

,index,visit_id,exposure_name,controller,day_obs,seq_num,physical_filter,band,s_ra,s_dec,...,solarElong,moonPhase,cummTelAz,observation_reason_opsim,science_program_opsim,start_date,HA,observationStartDatetime64,overhead,day_obs_iso8601
0,0,2024111500011,CC_O_20241115_000011,O,20241115,11,r_03,r,347.971741,-10.962135,...,110.770882,97.135491,NaN,None,None,2024-11-16 00:36:41.170,0.402312,2024-11-16 00:36:41.169763994,NaN,2024-11-15
1,1,2024111500012,CC_O_20241115_000012,O,20241115,12,r_03,r,347.971741,-10.962135,...,110.769282,97.127591,NaN,None,None,2024-11-16 00:38:59.123,0.440737,2024-11-16 00:38:59.122878760,132.948684,2024-11-15
2,2,2024111500018,CC_O_20241115_000018,O,20241115,18,r_03,r,347.971741,-10.962135,...,110.762379,97.093579,NaN,INFOCUS_SITCOM-826,BLOCK-T249,2024-11-16 00:48:41.677,0.603001,2024-11-16 00:48:41.677065315,577.555745,2024-11-15
3,3,2024111500025,CC_O_20241115_000025,O,20241115,25,r_03,r,347.971741,-10.962135,...,110.757568,97.069935,NaN,INFOCUS_SITCOM-826,BLOCK-T249,2024-11-16 00:55:36.429,0.718525,2024-11-16 00:55:36.428577499,384.747798,2024-11-15
4,4,2024111500031,CC_O_20241115_000031,O,20241115,31,r_03,r,347.971741,-10.962135,...,110.753536,97.050160,NaN,INFOCUS_SITCOM-826,BLOCK-T249,2024-11-16 01:01:23.995,0.815335,2024-11-16 01:01:23.995028003,317.569101,2024-11-15


Note that `read_consdb` **does support** maf stackers.

### Generalize visit reading for a night

A common interface to collect data for one night from either a baseline simulation or the ConsDB, depending on provided parameters, simplifies the composition of notebooks that support the user specifying either:

To get visits from `lsstcomcam` from the ConsDB:

In [25]:
visit_source = "lsstcomcam"
visits = schedview.collect.visits.read_visits(
    "2024-11-15", visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
)
visits.head()

,index,visit_id,exposure_name,controller,day_obs,seq_num,physical_filter,band,s_ra,s_dec,...,solarElong,moonPhase,cummTelAz,observation_reason_opsim,science_program_opsim,start_date,HA,observationStartDatetime64,overhead,day_obs_iso8601
0,0,2024111500011,CC_O_20241115_000011,O,20241115,11,r_03,r,347.971741,-10.962135,...,110.770882,97.135491,NaN,None,None,2024-11-16 00:36:41.170,0.402312,2024-11-16 00:36:41.169763994,NaN,2024-11-15
1,1,2024111500012,CC_O_20241115_000012,O,20241115,12,r_03,r,347.971741,-10.962135,...,110.769282,97.127591,NaN,None,None,2024-11-16 00:38:59.123,0.440737,2024-11-16 00:38:59.122878760,132.948684,2024-11-15
2,2,2024111500018,CC_O_20241115_000018,O,20241115,18,r_03,r,347.971741,-10.962135,...,110.762379,97.093579,NaN,INFOCUS_SITCOM-826,BLOCK-T249,2024-11-16 00:48:41.677,0.603001,2024-11-16 00:48:41.677065315,577.555745,2024-11-15
3,3,2024111500025,CC_O_20241115_000025,O,20241115,25,r_03,r,347.971741,-10.962135,...,110.757568,97.069935,NaN,INFOCUS_SITCOM-826,BLOCK-T249,2024-11-16 00:55:36.429,0.718525,2024-11-16 00:55:36.428577499,384.747798,2024-11-15
4,4,2024111500031,CC_O_20241115_000031,O,20241115,31,r_03,r,347.971741,-10.962135,...,110.753536,97.050160,NaN,INFOCUS_SITCOM-826,BLOCK-T249,2024-11-16 01:01:23.995,0.815335,2024-11-16 01:01:23.995028003,317.569101,2024-11-15


To get visits from the 3.5 baseline:

In [26]:
visit_source = "3.5"
visits = schedview.collect.visits.read_visits(
    "2025-11-15", visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
)
visits.head()

,fieldRA,fieldDec,observationStartMJD,visitExposureTime,filter,rotSkyPos,rotSkyPos_desired,numExposures,airmass,seeingFwhm500,...,moonRA,moonDec,moonDistance,solarElong,moonPhase,HA,observationStartDatetime64,overhead,day_obs_iso8601,start_date
observationId,,,,,,,,,,,,,,,,,,,,,
122180,58.413556,-29.424197,60995.108950,29.2,r,-160.051247,-160.051247,2,1.156988,0.485592,...,186.949282,-4.492207,120.165144,131.404174,25.270118,-2.300843,2025-11-16 02:36:53.250150927,NaN,2025-11-15,2025-11-16 02:36:53.250150927+00:00
122181,60.175498,-27.054031,60995.109391,29.2,r,-164.153629,-164.153629,2,1.175841,0.485592,...,186.956030,-4.494726,119.732488,133.456022,25.266375,-2.407687,2025-11-16 02:37:31.371335099,8.921184,2025-11-15,2025-11-16 02:37:31.371335099+00:00
122182,63.778641,-27.022417,60995.109836,29.2,r,-164.617202,-164.617202,2,1.216545,0.406227,...,186.962841,-4.497269,116.772743,132.711476,25.262597,-2.637177,2025-11-16 02:38:09.857544447,9.286209,2025-11-15,2025-11-16 02:38:09.857544447+00:00
122183,65.750661,-29.330384,60995.110279,29.2,r,-161.885202,-161.885202,2,1.231886,0.406227,...,186.969603,-4.499796,114.333273,129.993675,25.258846,-2.757999,2025-11-16 02:38:48.078959417,9.021415,2025-11-15,2025-11-16 02:38:48.078959417+00:00
122184,67.362068,-26.732113,60995.110720,29.2,r,-165.480220,-165.480220,2,1.262140,0.432611,...,186.976344,-4.502316,113.870453,131.973061,25.255107,-2.854810,2025-11-16 02:39:26.192080616,8.913121,2025-11-15,2025-11-16 02:39:26.192080616+00:00


## Displaying visits

### Text tables

The simplest way to show the visits table is using the standard `pandas` display function, but it is impracticle to display more than a small fraction of the table with it.
`schedview` provides an additional tabular display for visits built on `bokeh`s `DataTable` tool.
While the `DataTable` tool works in most notebook environments and within Times Square, it does not work correctly an the notebook aspect of the RSP (apparently due to a conflict with firefly).

In [27]:
%aimport schedview.examples.visittable
%psource schedview.examples.visittable

import pandas as pd
from bokeh.models.ui.ui_element import UIElement

import schedview.collect.visits
import schedview.plot
from schedview.dayobs import DayObs


def visit_table(
    visit_source: str = "3.5",
    day_obs: DayObs = DayObs.from_date("2026-03-15"),
    visible_columns: list[str] = ['observationStartMJD', 'fieldRA', 'fieldDec', 'filter']
) -> UIElement:
    visits: pd.DataFrame = schedview.collect.visits.read_visits(
        day_obs, visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
    )

    figure: UIElement = schedview.plot.create_visit_table(visits, visible_column_names=visible_columns, width=1024)
    return figure


In [28]:
p = schedview.examples.visittable.visit_table("3.5", "2025-11-15")
bokeh.io.show(p)

### Mapping the visits on the sky

In [29]:
%aimport schedview.examples.visitmap
%psource schedview.examples.visitmap

import pandas as pd
import uranography.api
from bokeh.models.ui.ui_element import UIElement
from rubin_scheduler.scheduler.model_observatory.model_observatory import ModelObservatory

import schedview.collect.visits
import schedview.compute.visits
import schedview.plot
from schedview.dayobs import DayObs


def visit_map(
    visit_source: str = "3.5",
    day_obs: DayObs = DayObs.from_date("2026-03-15"),
    nside: int = 8,
    map_classes=[uranography.api.ArmillarySphere],
) -> UIElement:

    # Collect the data to be shown
    visits: pd.DataFrame = schedview.collect.visits.read_visits(
        day_obs, visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
    )

    footprint = schedview.collect.footprint.get_footprint(nside)
    observatory: ModelObservatory = ModelObservatory(nside=nside, init_load_length=1)
    observatory.mjd = visits.observationStartMJD.max()
    conditions = observatory.return_conditions()

    # Do computations required by mapping code
    visits: pd

In [30]:
p = schedview.examples.visitmap.visit_map()
bokeh.io.show(p)

/sdf/data/rubin/user/neilsen/devel/rubin_scheduler/rubin_scheduler/skybrightness_pre/sky_model_pre.py:352: UserWarning: Requested MJD between sunrise and sunset, returning closest maps
  warnings.warn("Requested MJD between sunrise and sunset, returning closest maps")


## Sun and moon positions

In [37]:
%aimport schedview.examples.sunmoon
%psource schedview.examples.sunmoon

from schedview import DayObs
import schedview.compute.astro
import pandas as pd
from rubin_scheduler.scheduler.model_observatory import ModelObservatory

def sun_moon_positions(day_obs: DayObs = DayObs.from_date('2025-11-11')) -> str:
    # Get the time for which to get positions: the middle of the night
    # of the provided DayObs.
    night_events: pd.DataFrame = schedview.compute.astro.night_events(day_obs.date)
    mjd = night_events.loc["night_middle", "MJD"]

    observatory: ModelObservatory = ModelObservatory(init_load_length=1, mjd=mjd)
    positions: str = schedview.compute.astro.compute_sun_moon_positions(observatory)
    return positions


In [40]:
schedview.examples.sunmoon.sun_moon_positions()

coordinate,RA,dec,alt,az,phase
sun,227.227859,-17.650894,-41.997283,180.070590,NaN
moon,143.187410,17.673420,-13.765279,77.481798,49.929497


## Overhead summary table

In [41]:
%aimport schedview.examples.overheadtable
%psource schedview.examples.overheadtable

import pandas as pd

import schedview.collect.visits
import schedview.compute.astro
import schedview.compute.visits
import schedview.plot
from schedview import DayObs


def overhead_table(
    visit_source: str = "3.5",
    day_obs: DayObs = DayObs.from_date("2026-03-15"),
) -> str:
    # Get the time for which to get positions: the middle of the night
    # of the provided DayObs.
    night_events: pd.DataFrame = schedview.compute.astro.night_events(day_obs.date)

    # Get the visits
    visits: pd.DataFrame = schedview.collect.visits.read_visits(
        day_obs, visit_source, stackers=schedview.collect.visits.NIGHT_STACKERS
    )

    overhead_summary = schedview.compute.visits.compute_overhead_summary(
        visits,
        night_events.loc["sun_n12_setting", "MJD"],
        night_events.loc["sun_n12_rising", "MJD"],
    )
    summary_table = schedview.plot.create_overhead_summary_table(overhead_summary)
    return summary_table


In [46]:
summary_table = schedview.examples.overheadtable.overhead_table()
display(HTML(summary_table))

In [47]:
!pwd

/sdf/data/rubin/user/neilsen/devel/schedview/notebooks
